In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nibabel as nib
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf

In [ ]:
data = pd.read_csv('../input/covid19-ct-scans/metadata.csv')
data.head()

In [ ]:
def read_nii(filepath):
    '''
    Reads .nii file and returns pixel array
    '''
    ct_scan = nib.load(filepath)
    array   = ct_scan.get_fdata()
    array   = np.rot90(np.array(array))
    return(array)

In [ ]:
# Read sample
k = 19
sample_ct   = read_nii(data.loc[k,'ct_scan'])
sample_lung = read_nii(data.loc[k,'lung_mask'])
sample_infe = read_nii(data.loc[k,'infection_mask'])
sample_all  = read_nii(data.loc[k,'lung_and_infection_mask'])

In [ ]:
sample_all.shape

In [ ]:
n =sample_all.shape[2] % 2
n = 40

fig = plt.figure(figsize = (18,15))
plt.subplot(1,4,1)
plt.imshow(sample_ct[..., n], cmap = 'bone')
plt.title('Original Image')

plt.subplot(1,4,2)
plt.imshow(sample_ct[..., n], cmap = 'bone')
plt.imshow(sample_lung[..., n],alpha = 0.5, cmap = 'nipy_spectral')
plt.title('Lung Mask')

plt.subplot(1,4,3)
plt.imshow(sample_ct[..., n], cmap = 'bone')
plt.imshow(sample_infe[..., n], alpha = 0.5, cmap = 'nipy_spectral')
plt.title('Infection Mask')

plt.subplot(1,4,4)
plt.imshow(sample_ct[..., n], cmap = 'bone')
plt.imshow(sample_all[..., n], alpha = 0.5, cmap = 'nipy_spectral')
plt.title('Lung and Infection Mask')

plt.show()


fig = plt.figure(figsize = (18,15))
plt.subplot(1,4,1)
plt.imshow(sample_ct[..., n], cmap = 'bone')
plt.title('Original Image')

plt.subplot(1,4,2)

plt.imshow(sample_lung[..., n])
plt.title('Lung Mask')

plt.subplot(1,4,3)

plt.imshow(sample_infe[..., n])
plt.title('Infection Mask')

plt.subplot(1,4,4)
plt.imshow(sample_all[..., n])
plt.title('Lung and Infection Mask')

In [ ]:
im_size = sample_ct[:,:,1].shape

Region of interest


- get area with same label
- set min-max coordinate

In [ ]:
dd = sample_all[:,:,:].sum(axis=2)
plt.imshow(dd/sample_all.shape[2])
plt.show()

d_size_interes = np.where(dd > 0)
print(np.min(d_size_interes,axis=1),np.max(d_size_interes,axis=1))
print(len(d_size_interes),d_size_interes[0].shape,d_size_interes[1].shape)

In [ ]:
[np.min(d_size_interes,axis=1),np.max(d_size_interes,axis=1)]

Get class label

In [ ]:
set(sample_all[:,:,n].reshape((im_size[0]*im_size[1])).tolist())

* 0 - background
* 1 - left
* 2 - right
* 3 - infect

In [ ]:
set(sample_infe[:,:,n].reshape((im_size[0]*im_size[1])).tolist())

* 0 - background
* 1 - infect

## Load Data

In [ ]:
import math

In [ ]:
from tensorflow.keras.utils import Sequence

class DataSequence(Sequence):
    """
    Keras Sequence object to train a model on a list of csv files
    data, - csv - data with file name information
    batch_size=1, - batch size
    out_chanel = 2, - chanel number
    w_size = 256 - window size, if w_size = 1, full size
    in_chanel - number of uf input image on sample
    
    """
    def __init__(self, data, batch_size=1, out_chanel = 1, in_chanel = 1, w_size = 256, mode='train'):
        """
        df = dataframe with 4 columns: the labels and a list of filenames
        """
        
        self.bsz = batch_size
        self.mode = mode
        self.ind = np.arange(batch_size)
        
        # Take labels and a list of image locations in memory
        self.data = data
        self.x1 = 0
        self.x2 = w_size
        self.y1 = 0
        self.y2 = w_size
        self.w = w_size
        self.chanel = out_chanel
        self.seqenc = in_chanel // 2
        

    def __len__(self):
        return int(math.ceil((self.data.shape[0]) ))

    def on_epoch_end(self):
        #print('epoch end:')
        self.indexes = range(self.data.shape[0])
        if self.mode == 'train':
            # Shuffles indexes after each epoch if in training mode
            self.indexes = np.random.choice(self.indexes, size=len(self.indexes))
            #print('gen end:')
            

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        #idx * self.bsz: (idx + 1) * self.bsz
        #print(idx)
        imag = read_nii(data.loc[idx,'lung_and_infection_mask'])
        batch =[]
        #print(imag.shape)
        if self.w >1:
            # RoI
            d_size_interes = np.where(imag.sum(axis=2) > 0)
            # choice corner on new select RoI
            xy_min = np.min(d_size_interes,axis=1)
            xy_max = np.max(d_size_interes,axis=1)
            # randon corner
            conertb = np.random.choce(np.arange(2), size=1)
            conerrl = np.random.choce(np.arange(2), size=1)
            self.x1 = xy_min[1]+(xy_max[1]-xy_min[1]-self.w)*conerrl
            self.x2 = self.w + self.x1
            self.y1 = xy_min[0]+(xy_max[0]-xy_min[0]-self.w)*conertb
            self.y2 = self.w + self.y1
        else:
            self.x1 = 0 # full size
            self.x2 = imag.shape[1]
            self.y1 = 0
            self.y2 = imag.shape[0]
            
        # chanel on label
        for i in range(imag.shape[2]):
            
            imag4 = np.zeros((imag.shape[0],imag.shape[1],self.chanel))
            #print(imag4.shape,imag.shape)
            if self.chanel == 1:
                imag4[:,:,0] = imag[:,:,i]==3 # only infect
            if self.chanel == 3:
                imag4[:,:,0] = imag[:,:,i]==0 # all classes
                imag4[:,:,1] = imag[:,:,i]==1 
                imag4[:,:,1] = imag[:,:,i]==2 
                imag4[:,:,2] = imag[:,:,i]==3               
            if self.chanel == 2:
                imag4[:,:,0] = imag[:,:,i]==1 # infect + lung
                imag4[:,:,0] = imag[:,:,i]==2 
                imag4[:,:,1] = imag[:,:,i]==3
            # batch RoI
            batch.append(imag4[self.y1:self.y2,self.x1:self.x2,:])
        self.ind = np.random.choice(np.arange(1,len(batch)-1), size=self.bsz )
        #print(type(self.ind))
        # sampling labels to batch
        self.segment = np.array(batch)[self.ind].astype(float)    
        return self.segment

    def get_batch_features(self, idx):
        # Fetch a batch of inputs
        imag = read_nii(data.loc[idx,'ct_scan'])
        imag = imag.transpose([2,0,1])
        imag = imag / np.max(imag)
        # RoI on image + sampling
        imag =imag.reshape(imag.shape[0],imag.shape[1],imag.shape[2],1)
        return imag[self.ind,self.y1:self.y2,self.x1:self.x2,:]

    def __getitem__(self, idx):
        
        batch_y = self.get_batch_labels(idx)
        batch_x = self.get_batch_features(idx)
        return batch_x, batch_y
    
    
  

test my generator

In [ ]:
data_seq = DataSequence(data.iloc[:-2,:],50)
data_seq_test = DataSequence(data.iloc[-2:,:],10)
for i in range(2):
    x,y = data_seq[i]
    print(i,':',x.shape,y.shape)


In [ ]:
n = 2
fig = plt.figure(figsize = (15,5))
plt.subplot(1,3,1)
plt.imshow(x[n,:,:,0])
plt.subplot(1,3,2)
plt.imshow(y[n,:,:,0])
plt.subplot(1,3,3)
plt.imshow(x[n,:,:,0], cmap = 'bone')
plt.imshow(y[n,:,:,0], alpha = 0.5, cmap = 'nipy_spectral')

plt.show()

In [ ]:
np.max(x)

Buil UNet model

In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
import cv2
import skimage.color
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image
import tensorflow as tf
from keras import backend as K
from PIL import Image
import tensorflow
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, BatchNormalization, Activation, Dense, Dropout

#
from tensorflow.keras.layers import Conv2D, Conv2DTranspose
from tensorflow.keras.layers import MaxPooling2D, GlobalMaxPool2D
from tensorflow.keras.layers  import concatenate, add
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [ ]:
from tensorflow.python.framework import ops

from tensorflow.python.keras.utils import losses_utils

from tensorflow.python.ops import array_ops
from tensorflow.python.ops import math_ops

In [ ]:
def dice_coef(y_true, y_pred):
    return (2. * K.sum(y_true * y_pred) + 1.) / (K.sum(y_true) + K.sum(y_pred) + 1.)


def focal(y_true, y_pred):
    alpha = 1
    gamma = 2
    
    num = y_true.shape
    print(y_true,y_pred)

    
    y_pred = ops.convert_to_tensor_v2_with_dispatch(y_pred)
    y_true = math_ops.cast(y_true, y_pred.dtype)
    BCE_loss = tensorflow.keras.losses.CategoricalCrossentropy()

    return K.mean((y_pred - 1)** gamma * alpha * BCE_loss( y_true, y_pred), axis=-1)
   





In [ ]:

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
mm = VGG16(input_shape=(256,256,3), include_top=False, weights="imagenet")
mm.summary()
        

In [ ]:
len(mm.layers)

In [ ]:
class Gray2VGGInput( tensorflow.keras.layers.Layer ) :
    """Custom conversion layer
    """
    def build( self, x ) :
        self.image_mean = K.variable(value=np.array([103.939, 116.779, 123.68]).reshape([1,1,1,3]).astype('float32'), 
                                     dtype='float32', 
                                     name='imageNet_mean' )
        self.built = True
        return
    def call( self, x ) :
        rgb_x = K.concatenate( [x,x,x], axis=-1 )
        #norm_x = rgb_x - self.image_mean
        return rgb_x
    def compute_output_shape( self, input_shape ) :
        return input_shape[:3] + (3,)

In [ ]:
from tensorflow.keras.models import Sequential

In [ ]:
def unet_pre_train(use_pretrain = True, num_classes = 13, input_shape= (200, 200, 3),level = 3,neuron = 16, lr=0.0001,b1 = 0.9, b2=0.9999):  
    img_input = Input(shape = input_shape)
    if use_pretrain :
        # грузим   VGG16
        x_in  = Gray2VGGInput( name='gray_to_rgb_norm')( img_input )
        model_vgg16_3 = VGG16(input_tensor = x_in, input_shape = (input_shape[0],input_shape[1],3),include_top=False, weights="imagenet")

            
            
        
        #y = pre_trained_model(x_in) 
        
        # замораживаем все слои
        for layer in model_vgg16_3.layers:
            layer.trainable = False
            
        print('x_in: ',model_vgg16_3.layers)
        
        #y_bloc = [y]
        
        
         
        #print(x_in)
        #y = pre_trained_model.layers[0](x_in )
        #print(y)
        # фиксируем скипы для переноса из VGG16 (надо еще проверять)
        #blocks_ = [ pre_trained_model.layers[2].output]
        blocks_ =[ model_vgg16_3.layers[2].output]
        for i in range(level):
            #blocks_.append( pre_trained_model.layers[5+i*4].output)
            blocks_.append( model_vgg16_3.layers[5+i*4].output)
        #block_3_out = pre_trained_model.layers[6].output
        #block_2_out = pre_trained_model.layers[3].output
        
        #  фиксируем вход сети
        
        
        
        # фиксируем последний рабочий слой для своего потока  
        #x = pre_trained_model.layers[level*4+1].output
        x =  model_vgg16_3.layers[1+level*4].output
        #x = y_bloc[level*4+2]
        #print('y:',y_bloc)
        print('x:',x)
        
    else:
        #x = Conv2D(3, (3, 3), padding='same')(img_input )
        #pre_trained_model.layers[0].input = x                                      
        #x = pre_trained_model.layers[0].output
        x1 = img_input
        print(x1)
    
        blocks_ = []
        i = 0
        for i in range(level):
            x2 = Conv2D(neuron*(i+1), (3, 3), padding='same')(x1)
            
            x2 = BatchNormalization()(x2)
            x2 = Activation('relu')(x2)

            x2 = Conv2D(neuron*(i+1), (1, 1), padding='same')(x2)
            x2 = BatchNormalization()(x2)
            x2 = Activation('relu')(x2) 
            
            x3 = Conv2D(neuron*(i+1), (1, 1), padding='same')(x1)
            
            x3 = BatchNormalization()(x3)
            x3 = Activation('relu')(x3)

            x3 = Conv2D(neuron*(i+1), (3, 3), padding='same')(x3)
            x3 = BatchNormalization()(x3)
            x3 = Activation('relu')(x3)
            
            x4 = Conv2D(neuron*(i+1), (5, 5), padding='same')(x1)
            
            x4 = BatchNormalization()(x4)
            x4 = Activation('relu')(x4)

            
            x = concatenate([x2,x3,x4,x1], axis = 3 )
        
            blocks4 = x
            
            blocks_.append(x)
            
    
            # down i
            x = MaxPooling2D(padding='same')(x)
            x1 = x
            print('i: ',i,x)
    
     
    print('x up:',x)
    x = Conv2D(neuron*(level+1), (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(neuron*(level+1), (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # задаем список скипов
    print(blocks_)
    print(x)

    for i in range(level-1,0,-1):
      # UP i
      print(i,'x up:',x)
      x = Conv2DTranspose(neuron*i, (2, 2), strides=(2, 2), padding='same')(x)
      x = BatchNormalization()(x)
      x = Activation('relu')(x)
        
      x = concatenate([x, blocks_[i]] ) # добавили перенос из понижаюшего плеча 
      x = Conv2D(neuron*i, (3, 3), padding='same')(x)
      x = BatchNormalization()(x)
      x = Activation('relu')(x)

      x = Conv2D(neuron*i, (3, 3), padding='same')(x)
      x = BatchNormalization()(x)
      x = Activation('relu')(x)


    print(x)
    x = Conv2DTranspose(neuron*i, (2, 2), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    print(-1+i,'x up:',x)    
    x = concatenate([x, blocks_[0]] ) # добавили перенос из понижаюшего плеча 
    # последний слой сверток для классификации
    x = Conv2D(neuron, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    if num_classes>1:
      x = Conv2D(num_classes, (3, 3), activation='softmax', padding='same')(x)
      # собираем модель 
      print('model:',img_input,x)  
      model = Model(img_input, x)
      model.compile(optimizer=Adam(),
                    loss= 'categorical_crossentropy',#focal, #'
                    metrics=[dice_coef])
    else:
      x = Conv2D(num_classes, (3, 3), activation='sigmoid', padding='same')(x)
      # собираем модель
      print('model:',img_input,x) 
      model = Model(img_input, x)
      model.compile(optimizer=Adam(learning_rate=lr, beta_1=b1, beta_2=b2),
                    loss = 'mse',
                    metrics=[dice_coef])

    model.summary()
    # вернем модель
    return model

In [ ]:
im_size

In [ ]:
K.clear_session()
im_size=x[0,:,:,:].shape
model= unet_pre_train(use_pretrain = False,num_classes = 1, input_shape= (im_size[0], im_size[1], im_size[2]), level = 4,neuron = 16)

In [ ]:
from tensorflow.keras.utils import plot_model


In [ ]:
plot_model(model,'model1.png')

In [ ]:
es = EarlyStopping(monitor = ['val_loss'], patience = 3)
modch = ModelCheckpoint(monitor='val_loss',mode='min',save_best_only=True,save_weights_only=True,verbose=1,filepath='model.{epoch:02d}-{val_loss:.2f}.h5')

In [ ]:
#model.load_weights('./model.hdf5')


In [ ]:
history1 = model.fit_generator(data_seq, epochs=500, verbose=1,validation_data = data_seq_test) # , callbacks =[es,modch]

In [ ]:
model.save_weights('model.h5')

model.save_weights('model.HDF5')
model.save_weights('model.hdf5')

In [ ]:
first = False #reset history . If I want long history first = False 

In [ ]:
if first:
    history = history1
else:
    history.history['dice_coef'] = history.history['dice_coef']+history1.history['dice_coef']
    history.history['val_dice_coef'] = history.history['val_dice_coef']+history1.history['val_dice_coef']
    history.history['loss'] = history.history['loss']+history1.history['loss']
    history.history['val_loss'] = history.history['val_loss']+history1.history['val_loss']

In [ ]:
plt.plot(history.history['dice_coef'])
plt.plot(history.history['val_dice_coef'])
plt.title('dice vs Epochs')
plt.xlabel('Epochs')
plt.ylabel('dice')
plt.legend(['Train', 'Val'], loc = 'upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Loss vs Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Train', 'Val'], loc = 'upper right')
plt.show()

predicted masks

In [ ]:
X,Y = data_seq[0]
Xt,Yt = data_seq_test[0]

In [ ]:
predictedt = model.predict(Xt)
n = 0
chanel = 0
fig = plt.figure(figsize = (18,15))

plt.subplot(1,3,1)
plt.imshow(Xt[n][...,0], cmap = 'bone')
plt.title('original lung')

plt.subplot(1,3,2)
#plt.imshow(Xt[n][...,0], cmap = 'bone')
plt.imshow(Yt[n][...,chanel],alpha = 0.9, cmap = "nipy_spectral")
plt.title('original infection mask')

plt.subplot(1,3,3)
#plt.imshow(Xt[n][...,0], cmap = 'bone')
plt.imshow((predictedt[n,:,:,chanel]>predictedt[n,:,:,chanel].mean()*5.0).astype(float),alpha = 0.9,cmap = "nipy_spectral")
plt.title('predicted infection mask')
plt.show()

откалибруем ответ по тренировочному набору

просто по несмкольким примерам подбираю порог (можно и получше сдедать) - predicted[n,:,:,chanel]>predicted[n,:,:,chanel].mean()*5.0

In [ ]:
predicted = model.predict(X)
n = 10
chanel = 0
fig = plt.figure(figsize = (18,15))

plt.subplot(1,3,1)
plt.imshow(X[n][...,0], cmap = 'bone')
plt.title('original lung')

plt.subplot(1,3,2)
#plt.imshow(X[n][...,0], cmap = 'bone')
plt.imshow(Y[n][...,chanel],alpha = 0.9, cmap = "nipy_spectral")
plt.title('original infection mask')

plt.subplot(1,3,3)
#plt.imshow(X[n][...,0], cmap = 'bone')
plt.imshow((predicted[n,:,:,chanel]>predicted[n,:,:,chanel].mean()*5.).astype(float),alpha = 0.9,cmap = "nipy_spectral")
plt.title('predicted infection mask')
plt.show()

Полный размер

In [ ]:
# Read sample - генератор больших картинок
data_seq_test_big_size = DataSequence(data.iloc[-2:,:],20,w_size = 1)
for i in range(2):
    x,y = data_seq_test_big_size[i]
    print(i,':',x.shape,y.shape)


Создаю модель для полного размера объектовЖ

- строим размер изображения по объекту из генератора (512х512)
- Создаем сеть UNet+inception для картинки 512х512
- грузим в нее веса маленькой сети model
- запускаем визуализацию ответов
- результат не без огрех, но пойдет

In [ ]:
# create big model



im_size=x[0,:,:,:].shape
model_big_size= unet_pre_train(use_pretrain = False,num_classes = 1, input_shape= (im_size[0], im_size[1], im_size[2]), level = 4,neuron = 16)


In [ ]:
# loaw weight from model
model_big_size.load_weights('model.h5')

In [ ]:
#test
Xtb,Ytb = data_seq_test_big_size[0]
predictedtb = model_big_size.predict(Xtb)
for i in range(10):
    n = i
    chanel = 0
    fig = plt.figure(figsize = (18,15))

    plt.subplot(1,3,1)
    plt.imshow(Xtb[n][...,0], cmap = 'bone')
    plt.title('original lung')

    plt.subplot(1,3,2)
    #plt.imshow(X[n][...,0], cmap = 'bone')
    plt.imshow(Ytb[n][...,chanel],alpha = 0.9, cmap = "nipy_spectral")
    plt.title('original infection mask')

    plt.subplot(1,3,3)
    #plt.imshow(X[n][...,0], cmap = 'bone')
    plt.imshow((predictedtb[n,:,:,chanel]>predictedtb[n,:,:,chanel].mean()*5.).astype(float),alpha = 0.9,cmap = "nipy_spectral")
    plt.title('predicted infection mask')
    plt.show()

In [ ]:
model_big_size.evaluate_generator(data_seq_test_big_size,verbose=1)


Пока не очень - нужно учить